This extension serves a narrow use case for writing, compiling, error-checking and validating stan models within jupyter notebook. I am not a comfortable R user and STAN currently leans towards R. I wrote this simple script as I wanted an uninterruped workflow for data manipulation, stan model creation and analysis  all in jupyter notebook. Once the codemirror support is merged in, sysntax highlighting for stan code could also be enabled. Since  few people were asking me for this, I have packaged this as a extension.

Tested to work on Linux/Mac. Python versions 2.7/3.6

## Installation

`pip install git+https://github.com/Arvinds-ds/stanmagic.git`

## Testing %%stan magic

In [1]:
import numpy as np
from collections import OrderedDict
import pystan

### Load stanmagic extension

In [2]:
%load_ext stanmagic 

#### Optional: Using stanc compiler instead of pystan.stanc compiler

To get cmdstan installed:-

1. Downlad latest cmdstan-xxxx.tar.gz https://github.com/stan-dev/cmdstan/releases
2. extract to <cmdstan_path>
3. `cd <cmdstan_path>`
4. `make`
3. `make build`
4. export PATH=$PATH:<cmdstan_path>/bin/stanc


If the compiler is not in your path or you don't want to edit your PATH, you may have to pass --stanc [compiler_path] to %%stan. See below

### Generate test data

In [3]:
X = np.random.randn(100,3)
beta = np.array([0.1,0.2,0.3])
alpha = 4
sigma = 1.7
y = np.dot(X,beta) + alpha + np.random.randn(100)*sigma
N=100
K=3

In [4]:
data = OrderedDict({'X':X, 'y': y, 'N':N, 'K': K})

### 1. %%stan -f [stan_file_name]

Saves the cell code to a file specified in [stan_file_name]. The file name can also be accessed in __`_stan_model.model_file`__ generated in local namespace

In [5]:
%%stan -f test.stan
data {
  
  int<lower=1> N;
  int<lower=1> K;
  
  matrix[N,K] X;
  vector[N] y;
}

parameters{
  real alpha;
  ordered[K] beta;
  real<lower=0> sigma;
  
}

model {
  
  alpha ~ normal(0,10);
  beta[1] ~ normal(.1,.1);
  beta[2] ~ normal(.2,.1);
  beta[3] ~ normal(.3,.1);
  sigma ~ exponential(1);
  for (n in 1:N) {
    y[n] ~ normal(X[n]*beta + alpha, sigma);
  } 
  
}

generated quantities {
  vector[N] y_rep;
  for (n in 1:N) {
    y_rep[n] = normal_rng(X[n]*beta + alpha, sigma);
  }
}

Using pystan.stanc compiler..
-------------------------------------------------------------------------------
Model compiled successfully. Output stored in _stan_model object.
Type _stan_model in a cell to see a nicely formatted code output in a notebook
     ^^^^^^^^^^^
Access model compile output properties
_stan_model.model_file -> Name of stan_file [test.stan]
_stan_model.model_name -> Name of stan model [test_model]
_stan_model.model_code -> Model code [data {      int<lowe ....]


In [6]:
_stan_model

model_file: test.stan
model_name: test_model
model_code:
data {
  
  int<lower=1> N;
  int<lower=1> K;
  
  matrix[N,K] X;
  vector[N] y;
}

parameters{
  real alpha;
  ordered[K] beta;
  real<lower=0> sigma;
  
}

model {
  
  alpha ~ normal(0,10);
  beta[1] ~ normal(.1,.1);
  beta[2] ~ normal(.2,.1);
  beta[3] ~ normal(.3,.1);
  sigma ~ exponential(1);
  for (n in 1:N) {
    y[n] ~ normal(X[n]*beta + alpha, sigma);
  } 
  
}

generated quantities {
  vector[N] y_rep;
  for (n in 1:N) {
    y_rep[n] = normal_rng(X[n]*beta + alpha, sigma);
  }
}

In [7]:
model = pystan.StanModel(file=_stan_model.model_file)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_b64da7aba54424736fda036b359c5ae9 NOW.


In [8]:
model.sampling(data=data)

Inference for Stan model: anon_model_b64da7aba54424736fda036b359c5ae9.
4 chains, each with iter=2000; warmup=1000; thin=1; 
post-warmup draws per chain=1000, total post-warmup draws=4000.

            mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
alpha        4.1  3.0e-3   0.16   3.77   3.99    4.1   4.21   4.43   2914    1.0
beta[0]     0.08  1.6e-3   0.07  -0.07   0.03   0.08   0.13   0.22   2206    1.0
beta[1]     0.23  1.0e-3   0.07    0.1   0.18   0.23   0.27   0.35   4000    1.0
beta[2]     0.33  1.1e-3   0.07    0.2   0.28   0.33   0.38   0.48   4000    1.0
sigma       1.62  2.3e-3   0.12   1.42   1.54   1.62    1.7   1.88   2685    1.0
y_rep[0]    4.21    0.03   1.64   0.98   3.12    4.2   5.31   7.35   3747    1.0
y_rep[1]     4.2    0.03   1.64   0.99   3.08   4.21   5.33   7.33   3728    1.0
y_rep[2]    4.43    0.03   1.65    1.2   3.35   4.41   5.49   7.71   3859    1.0
y_rep[3]    4.15    0.03   1.64   0.85   3.07   4.13   5.25   7.39   4000    1.0
y

### 2. %%stan -v model_test

Saves the cell code to a code string. The `-v` specifies the object name where the output object is saved.
The model code can  be accessed in __`model_test.model_code`__ generated in local namespace

In [9]:
%%stan -v model_test

data {
  
  int<lower=1> N;
  int<lower=1> K;
  
  matrix[N,K] X;
  vector[N] y;
}

parameters{
  real alpha;
  ordered[K] beta;
  real<lower=0> sigma;
  
}

model {
  
  alpha ~ normal(0,10);
  beta[1] ~ normal(.1,.1);
  beta[2] ~ normal(.2,.1);
  beta[3] ~ normal(.3,.1);
  sigma ~ exponential(1);
  for (n in 1:N) {
    y[n] ~ normal(X[n]*beta + alpha, sigma);
  } 
  
}

generated quantities {
  vector[N] y_rep;
  for (n in 1:N) {
    y_rep[n] = normal_rng(X[n]*beta + alpha, sigma);
  }
}

Using pystan.stanc compiler..
-------------------------------------------------------------------------------
Model compiled successfully. Output stored in model_test object.
Type model_test in a cell to see a nicely formatted code output in a notebook
     ^^^^^^^^^^
Access model compile output properties
model_test.model_file -> Name of stan_file [None]
model_test.model_name -> Name of stan model [None]
model_test.model_code -> Model code [ data {      int<low ....]


In [10]:
model_test

model_file: Undefined
model_name: Undefined
model_code:

data {
  
  int<lower=1> N;
  int<lower=1> K;
  
  matrix[N,K] X;
  vector[N] y;
}

parameters{
  real alpha;
  ordered[K] beta;
  real<lower=0> sigma;
  
}

model {
  
  alpha ~ normal(0,10);
  beta[1] ~ normal(.1,.1);
  beta[2] ~ normal(.2,.1);
  beta[3] ~ normal(.3,.1);
  sigma ~ exponential(1);
  for (n in 1:N) {
    y[n] ~ normal(X[n]*beta + alpha, sigma);
  } 
  
}

generated quantities {
  vector[N] y_rep;
  for (n in 1:N) {
    y_rep[n] = normal_rng(X[n]*beta + alpha, sigma);
  }
}

In [11]:
model = pystan.StanModel(model_code=model_test.model_code)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_aab594f261a4c970392202fe126f1e99 NOW.


### 3. %%stan -f test3.stan -v test3_model

In [12]:
%%stan -f test3.stan -v test3_model

data {
  
  int<lower=1> N;
  int<lower=1> K;
  
  matrix[N,K] X;
  vector[N] y;
}

parameters{
  real alpha;
  ordered[K] beta;
  real<lower=0> sigma;
  
}

model {
  
  alpha ~ normal(0,10);
  beta[1] ~ normal(.1,.1);
  beta[2] ~ normal(.2,.1);
  beta[3] ~ normal(.3,.1);
  sigma ~ exponential(1);
  for (n in 1:N) {
    y[n] ~ normal(X[n]*beta + alpha, sigma);
  } 
  
}

generated quantities {
  vector[N] y_rep;
  for (n in 1:N) {
    y_rep[n] = normal_rng(X[n]*beta + alpha, sigma);
  }
}

Using pystan.stanc compiler..
-------------------------------------------------------------------------------
Model compiled successfully. Output stored in test3_model object.
Type test3_model in a cell to see a nicely formatted code output in a notebook
     ^^^^^^^^^^^
Access model compile output properties
test3_model.model_file -> Name of stan_file [test3.stan]
test3_model.model_name -> Name of stan model [test3_model]
test3_model.model_code -> Model code [ data {      int<low ....]


In [13]:
test3_model

model_file: test3.stan
model_name: test3_model
model_code:

data {
  
  int<lower=1> N;
  int<lower=1> K;
  
  matrix[N,K] X;
  vector[N] y;
}

parameters{
  real alpha;
  ordered[K] beta;
  real<lower=0> sigma;
  
}

model {
  
  alpha ~ normal(0,10);
  beta[1] ~ normal(.1,.1);
  beta[2] ~ normal(.2,.1);
  beta[3] ~ normal(.3,.1);
  sigma ~ exponential(1);
  for (n in 1:N) {
    y[n] ~ normal(X[n]*beta + alpha, sigma);
  } 
  
}

generated quantities {
  vector[N] y_rep;
  for (n in 1:N) {
    y_rep[n] = normal_rng(X[n]*beta + alpha, sigma);
  }
}

In [14]:
model = pystan.StanModel(file=test3_model.model_file)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_aab594f261a4c970392202fe126f1e99 NOW.


### 3. %%stan -f [stan_file_name] --save_only

Saves the cell code to a file specified in [stan_file_name].  Skips compile step

In [15]:
%%stan -f test1.stan --save_only
data {
  
  int<lower=1> N;
  int<lower=1> K;
  
  matrix[N,K] X;
  vector[N] y;
}

parameters{
  real alpha;
  ordered[K] beta;
  real<lower=0> sigma;
  
}

model {
  
  alpha ~ normal(0,10);
  beta[1] ~ normal(.1,.1);
  beta[2] ~ normal(.2,.1);
  beta[3] ~ normal(.3,.1);
  sigma ~ exponential(1);
  for (n in 1:N) {
    y[n] ~ normal(X[n]*beta + alpha, sigma);
  } 
  
}

generated quantities {
  vector[N] y_rep;
  for (n in 1:N) {
    y_rep[n] = normal_rng(X[n]*beta + alpha, sigma);
  }
}

File test1.stan saved..Skipping Compile


In [16]:
model = pystan.StanModel(file='test1.stan')

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_b64da7aba54424736fda036b359c5ae9 NOW.


### 4. %%stan 

Saves the cell code to a code string. The code string can be accessed via __`_stan_model.model_code`__

In [17]:
%%stan
data {
  
  int<lower=1> N;
  int<lower=1> K;
  
  matrix[N,K] X;
  vector[N] y;
}

parameters{
  real alpha;
  ordered[K] beta;
  real<lower=0> sigma;
  
}

model {
  
  alpha ~ normal(0,10);
  beta[1] ~ normal(.1,.1);
  beta[2] ~ normal(.2,.1);
  beta[3] ~ normal(.3,.1);
  sigma ~ exponential(1);
  for (n in 1:N) {
    y[n] ~ normal(X[n]*beta + alpha, sigma);
  } 
  
}

generated quantities {
  vector[N] y_rep;
  for (n in 1:N) {
    y_rep[n] = normal_rng(X[n]*beta + alpha, sigma);
  }
}

Using pystan.stanc compiler..
-------------------------------------------------------------------------------
Model compiled successfully. Output stored in _stan_model object.
Type _stan_model in a cell to see a nicely formatted code output in a notebook
     ^^^^^^^^^^^
Access model compile output properties
_stan_model.model_file -> Name of stan_file [None]
_stan_model.model_name -> Name of stan model [None]
_stan_model.model_code -> Model code [data {      int<lowe ....]


In [18]:
_stan_model.model_code

data {
  
  int<lower=1> N;
  int<lower=1> K;
  
  matrix[N,K] X;
  vector[N] y;
}

parameters{
  real alpha;
  ordered[K] beta;
  real<lower=0> sigma;
  
}

model {
  
  alpha ~ normal(0,10);
  beta[1] ~ normal(.1,.1);
  beta[2] ~ normal(.2,.1);
  beta[3] ~ normal(.3,.1);
  sigma ~ exponential(1);
  for (n in 1:N) {
    y[n] ~ normal(X[n]*beta + alpha, sigma);
  } 
  
}

generated quantities {
  vector[N] y_rep;
  for (n in 1:N) {
    y_rep[n] = normal_rng(X[n]*beta + alpha, sigma);
  }
}

In [19]:
model = pystan.StanModel(model_code=_stan_model.model_code)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_b64da7aba54424736fda036b359c5ae9 NOW.


In [20]:
model.sampling(data=data)

Inference for Stan model: anon_model_b64da7aba54424736fda036b359c5ae9.
4 chains, each with iter=2000; warmup=1000; thin=1; 
post-warmup draws per chain=1000, total post-warmup draws=4000.

            mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
alpha        4.1  3.1e-3   0.16   3.78   3.99   4.09   4.21   4.41   2829    1.0
beta[0]     0.08  1.7e-3   0.08  -0.08   0.03   0.08   0.13   0.22   2113    1.0
beta[1]     0.22  1.1e-3   0.07   0.09   0.18   0.23   0.27   0.35   3437    1.0
beta[2]     0.33  1.2e-3   0.07   0.19   0.28   0.33   0.37   0.48   4000    1.0
sigma       1.62  2.0e-3   0.11   1.42   1.55   1.62   1.69   1.87   3312    1.0
y_rep[0]    4.24    0.03   1.65   1.02   3.17   4.22   5.35    7.5   4000    1.0
y_rep[1]     4.2    0.03   1.66   0.84   3.08   4.19   5.33   7.41   3682    1.0
y_rep[2]    4.37    0.03   1.62   1.19   3.27   4.41   5.44   7.58   3901    1.0
y_rep[3]    4.15    0.03   1.65   0.93    3.0   4.14   5.27   7.38   4000    1.0
y

### 5. %%stan --stanc pystan

In [21]:
%%stan --stanc pystan
data {
  
  int<lower=1> N;
  int<lower=1> K;
  
  matrix[N,K] X;
  vector[N] y;
}

parameters{
  real alpha;
  ordered[K] beta;
  real<lower=0> sigma;
  
}

model {
  
  alpha ~ normal(0,10);
  beta[1] ~ normal(.1,.1);
  beta[2] ~ normal(.2,.1);
  beta[3] ~ normal(.3,.1);
  sigma ~ exponential(1);
  for (n in 1:N) {
    y[n] ~ normal(X[n]*beta + alpha, sigma);
  } 
  
}

generated quantities {
  vector[N] y_rep;
  for (n in 1:N) {
    y_rep[n] = normal_rng(X[n]*beta + alpha, sigma);
  }
}

Using pystan.stanc compiler..
-------------------------------------------------------------------------------
Model compiled successfully. Output stored in _stan_model object.
Type _stan_model in a cell to see a nicely formatted code output in a notebook
     ^^^^^^^^^^^
Access model compile output properties
_stan_model.model_file -> Name of stan_file [None]
_stan_model.model_name -> Name of stan model [None]
_stan_model.model_code -> Model code [data {      int<lowe ....]


In [22]:
model = pystan.StanModel(model_code=_stan_model.model_code)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_b64da7aba54424736fda036b359c5ae9 NOW.


### 6. %%stan --stanc "~/Downloads/cmdstan-2.16.0/bin/stanc"

In [23]:
%%stan --stanc "~/Downloads/cmdstan-2.16.0/bin/stanc"
data {
  
  int<lower=1> N;
  int<lower=1> K;
  
  matrix[N,K] X;
  vector[N] y;
}

parameters{
  real alpha;
  ordered[K] beta;
  real<lower=0> sigma;
  
}

model {
  
  alpha ~ normal(0,10);
  beta[1] ~ normal(.1,.1);
  beta[2] ~ normal(.2,.1);
  beta[3] ~ normal(.3,.1);
  sigma ~ exponential(1);
  for (n in 1:N) {
    y[n] ~ normal(X[n]*beta + alpha, sigma);
  } 
  
}

generated quantities {
  vector[N] y_rep;
  for (n in 1:N) {
    y_rep[n] = normal_rng(X[n]*beta + alpha, sigma);
  }
}

Using stanc compiler:  ~/Downloads/cmdstan-2.16.0/bin/stanc
~/Downloads/cmdstan-2.16.0/bin/stanc --o=/tmp/anon_e6a5c15a-a0a7-469e-99ae-a32d6f072fc6_model.cpp /tmp/anon_e6a5c15a-a0a7-469e-99ae-a32d6f072fc6.stan
Model name=anon_e6a5c15a_a0a7_469e_99ae_a32d6f072fc6_model
Input file=/tmp/anon_e6a5c15a-a0a7-469e-99ae-a32d6f072fc6.stan
Output file=/tmp/anon_e6a5c15a-a0a7-469e-99ae-a32d6f072fc6_model.cpp
-------------------------------------------------------------------------------
Model compiled successfully. Output stored in _stan_model object.
Type _stan_model in a cell to see a nicely formatted code output in a notebook
     ^^^^^^^^^^^
Access model compile output properties
_stan_model.model_file -> Name of stan_file [None]
_stan_model.model_name -> Name of stan model [None]
_stan_model.model_code -> Model code [data {      int<lowe ....]


In [24]:
model = pystan.StanModel(model_code=_stan_model.model_code)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_b64da7aba54424736fda036b359c5ae9 NOW.


### 7. %%stan -f [stan_file_name] -o [cpp_file_name]
Saves the cell code to a file specified in [stan_file_name] and outputs the compiled cpp file to the file name specified by [cpp_file_name]


### 8. %% stan -f  [stan_file_name] --allow_undefined
passes the --allow_undefined argument to stanc compiler

### 9.%%stan -f  [stan_file_name] --stanc [stanc_compiler_path]
Saves the cell code to a file specified in [stan_file_name] and compiles using the stan compiler specified in [stanc_compiler]. By default, it uses stanc compiler in your path. If your path does not have the stanc compiler, use this option (e.g %%stan binom.stan --stanc "~/cmdstan-2.16.0/bin/stanc")

In [25]:
assert True